# Esse notebook é uma continuação do Competição Kaggle - Titanic - Data Understanding

## Dentro desse segundo caderno, vamos começar a desenvolver o modelo de machine learning e criar o dataframe para submissão do desafio

In [1]:
#Criação e manipulação dos dataframes
import pandas as pd 

#Operações matemáticas
import numpy as np 

#Biblioteca de normalização
from sklearn.preprocessing import StandardScaler, RobustScaler

#Classificadores
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import Perceptron, SGDClassifier, LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#Separar os dataframes em treino e teste
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

#Métricas de qualidade do modelo 
from sklearn.metrics import accuracy_score

#Verificar o run time dos modelos
from time import time

#Biblioteca de AUTOML do sklearn 
from autosklearn.classification import AutoSklearnClassifier

ModuleNotFoundError: No module named 'autosklearn'

## Importando os dados e realizando o tratamento deles

In [ ]:
df_train = pd.read_csv('Data/train.csv')
df_test = pd.read_csv('Data/test.csv')

def limpar_dataframe(df):
    #Criando a nova coluna somando os irmãos/conjugues e pais/filhos
    df['Tamanho_Familia'] = df['SibSp'] + df['Parch']
    
    #Criando uma nova coluna para verificar quem estava sozinho
    df['Sozinho'] = df['Tamanho_Familia'].apply(lambda x: 1 if x == 0 else 0)
    
    #Completando os valores de embarque com a moda da coluna
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    
    #Completando os valores de idade e preço com a mediana da coluna
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    
    #Pegando apenas a letra da cabine
    df['Letra_Cabine'] = df['Cabin'].str[:1]
    
    #Criando uma coluna com apenas o titulo da pessoa
    df['Titulo'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    #Pegando os titulos mais comuns e retirando o resto
    titulos = df['Titulo'].value_counts(ascending=False)[:4]
    #Deixando os titulos com poucos valores como Misc
    df['Titulo'] = df['Titulo'].apply(lambda x: x if x in titulos else 'Misc')
    
    #Transformando as colunas categóricas em numéricas
    #df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Letra_Cabine', 'Titulo'])
    
    #Removendo as colunas que não serão necessárias
    df.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'], inplace=True)
    
limpar_dataframe(df_train)
limpar_dataframe(df_test)

df_train = pd.get_dummies(df_train, columns=['Titulo', 'Embarked', 'Sex', 'Letra_Cabine'])
df_test = pd.get_dummies(df_test, columns=['Titulo', 'Embarked', 'Sex', 'Letra_Cabine'])

## Separando em treino/teste

In [ ]:
X = df_train.drop(columns=['Survived'])
y = df_train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.50, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_validation.shape)
print(y_validation.shape)

## Testando os diferentes classificadores

In [ ]:
models = [
    RandomForestClassifier(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    Perceptron(),
    MLPClassifier(),
    SVC(),
    GaussianNB(),
    DecisionTreeClassifier(),
    XGBClassifier(),
    CatBoostClassifier(verbose=0),
    SGDClassifier(),
    LinearSVC(),
    LogisticRegression(),
    
    
]

accuracy = []
name = []
parameters = []
tempo = []

for model in models:
    
    tempo_inicial = time()
    
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    accuracy.append(accuracy_score(y_test, y_pred))
    name.append(model.__class__.__name__)
    parameters.append(model.get_params())
    
    tempo_final = time() - tempo_inicial
    tempo.append(tempo_final)
    
resultado = pd.DataFrame({'Nome': name,
                         'Accuracy': accuracy,
                         'Run Time': tempo,
                         'Parameters': parameters})

resultado.sort_values(by='Accuracy', ascending=False)

## Realizando o tuning dos hiperparâmetros do Random Forest Classifier

In [ ]:
parameters = {
    'n_estimators': np.arange(100,2000,100),
    'max_features': ['sqrt', 'log2'],
    'max_depth': np.arange(2,20),
    'bootstrap': [True, False],
    'min_samples_leaf': [1,2,4],
    'min_samples_split': [2,5,10]
}

classifier = GridSearchCV(RandomForestClassifier(), parameters, n_jobs=-1, verbose=1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classifier.best_params_)

In [ ]:
parameters = {
    'n_estimators': np.arange(100,2000,100),
    'max_features': ['sqrt', 'log2'],
    'max_depth': np.arange(2,20),
    'bootstrap': [True, False],
    'min_samples_leaf': [1,2,4],
    'min_samples_split': [2,5,10]
    
}

classifier = RandomizedSearchCV(RandomForestClassifier(), parameters, n_iter=100,
                               verbose=2, n_jobs=-1, random_state=42)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classifier.best_params_)

## Tuning dos hiperparâmetros do CatBoost Classifier

#### Dicionário de cada valor ajustado pelo GridSearchCV 

iterations: número máximo de árvores que podem ser construídas <br>
learning_rate: velocidade que os pesos do modelo são atualizados após trabalhar em cada batch de amostras de treino [0.0 - 1.0] <br>
depth: profundidade da árvore, na maior parte dos casos, um bom valor está entre [1-10] <br>



In [ ]:
%%timeit
parameters = {'iterations': [300,500,1000],
             'learning_rate': [0.01,0.02],
             'depth': [6,7,9,10],
             'l2_leaf_reg': [1,3,5,7,9,10]}

classifier = GridSearchCV(CatBoostClassifier(verbose=0), parameters, n_jobs=-1, verbose=2)
classifier.fit(X_train, y_train, eval_set=(X_validation, y_validation), use_best_model=True)
y_pred = classifier.predict(X_test)

print(accuracy_score(y_test, y_pred))

In [ ]:
classifier.best_params_

In [ ]:
classifier = CatBoostClassifier(random_strength=1, verbose=0)

## Utilizando o auto-sklearn para testar diferentes modelos 